In [1]:
# COMP9517 Group Project

In [2]:
### Model 1 : KNN

# Please run this file after dataVisualisation.ipynb. Afterwards, you can run any model in any order.

In [3]:
import numpy as np
import cv2 as cv
from pycocotools.coco import COCO
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
import albumentations as A
import os
import matplotlib.pyplot as plt


c:\Users\61402\anaconda3\envs\cv\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
# Define transformation for resizing and normalizing
resize_transform = A.Compose([
    A.Resize(512, 512),  # Resize images to a consistent size
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalization
])

class SeaTurtleDatasetForKNN(Dataset):
    def __init__(self, image_ids, transform=None):
        self.coco = COCO("./turtles-data/data/updated_annotations.json")
        self.image_ids = image_ids
        self.cat_ids = self.coco.getCatIds()
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, index):
        # Load image
        image_id = self.image_ids[index]
        image_data = self.coco.loadImgs([image_id])[0]

        image_path = os.path.join("./turtles-data/data", image_data["file_name"])
        image = cv.imread(image_path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        # Prepare mask
        mask = self._getmask(image_id, image)

        # Apply transformations
        if self.transform is not None:
            augmented = self.transform(image=image)
            image = augmented["image"]
            mask = cv.resize(mask, (512, 512), interpolation=cv.INTER_NEAREST)

        # Flatten images and masks for KNN input
        return image.flatten(), mask.flatten()

    def _getmask(self, image_id, image):
        """
        Generate mask with labels for each category.
        """
        categories = {"turtle": 1, "flipper": 2, "head": 3}
        # Initialize the final mask with zeros
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        # Process each category
        for category_name, category_id in categories.items():
            ann_ids = self.coco.getAnnIds(imgIds=image_id, catIds=category_id, iscrowd=None)
            annotations = self.coco.loadAnns(ann_ids)

            # Create a temporary mask for the current category
            temp_mask = np.zeros_like(mask)
            for ann in annotations:
                temp_mask += self.coco.annToMask(ann)
            
            # Assign category-specific value to the final mask
            if category_name == "turtle":
                mask[temp_mask > 0] = 1
            elif category_name == "flipper":
                mask[temp_mask > 0] = 2
            elif category_name == "head":
                mask[temp_mask > 0] = 3

        return mask

In [5]:
# Load dataset
annotation_file = "./turtles-data/data/updated_annotations.json"
coco = COCO(annotation_file)
image_ids = coco.getImgIds()

loading annotations into memory...
Done (t=15.36s)
creating index...
index created!


In [6]:
# Split dataset into training and test sets
train_ids, test_ids = train_test_split(image_ids, test_size=0.1, random_state=42)

In [7]:
train_dataset = SeaTurtleDatasetForKNN(train_ids, transform=resize_transform)
test_dataset = SeaTurtleDatasetForKNN(test_ids, transform=resize_transform)

loading annotations into memory...
Done (t=17.76s)
creating index...
index created!
loading annotations into memory...
Done (t=90.87s)
creating index...
index created!


In [ ]:
# Prepare the data for KNN by flattening images and masks
X_train, y_train = [], []
for i in range(len(train_dataset)):
    img, mask = train_dataset[i]
    X_train.append(img)
    y_train.append(mask)


In [ ]:
X_test, y_test = [], []
for i in range(len(test_dataset)):
    img, mask = test_dataset[i]
    X_test.append(img)
    y_test.append(mask)


In [ ]:
# Convert to numpy arrays
X_train = np.array(X_train)
y_train = np.concatenate(y_train).astype(int)  # Flatten all masks into a single array for y_train
X_test = np.array(X_test)
y_test = np.concatenate(y_test).astype(int)  # Flatten all masks into a single array for y_test

In [ ]:
# Initialize and train KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

In [ ]:
# Predict and evaluate
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))